In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2

In [2]:
#x,y,c,
poseName = 'Second-Position'
Coords = np.load("RootCoordinates/"+poseName+"X.npy")
print(Coords)
print(len(Coords))

[[0.504087 0.276401 0.871369 ... 0.463214 0.753358 0.833894]
 [0.506103 0.273689 0.878322 ... 0.463261 0.750616 0.831335]
 [0.508091 0.271147 0.873738 ... 0.465214 0.750646 0.831586]
 ...
 [0.499861 0.311969 0.919666 ... 0.479427 0.726151 0.862607]
 [0.499852 0.311945 0.909916 ... 0.479413 0.728764 0.852494]
 [0.499854 0.311986 0.918177 ... 0.479388 0.726172 0.863274]]
869


In [3]:
#Helper Methods:
import math
def calcDistance(x1,y1,x2,y2):
    distance = math.sqrt((x2-x1)**2 + (y2-y1)**2)
    return distance

In [4]:
#Get the Mean Skeleton's coordinates
PoseMeanX = np.load("Stats/"+poseName+"MeanX.npy")
print(PoseMeanX)
print(len(PoseMeanX))
PoseMeanY = np.load("Stats/"+poseName+"MeanY.npy")
print(PoseMeanY)
print(len(PoseMeanY))
MeanTuples = list(zip(PoseMeanX, PoseMeanY))
print(MeanTuples)

[0.49845949 0.49914523 0.46491369 0.40809603 0.35535707 0.53422329
 0.5916942  0.64399317 0.5        0.47843354 0.4720732  0.45469956
 0.52314248 0.53192549 0.55277685 0.49126899 0.50597548 0.48134145
 0.51590193 0.58771645 0.58665533 0.54655197 0.42059691 0.42240306
 0.46007413]
25
[-0.28466157 -0.35277208 -0.35297508 -0.3689446  -0.37783721 -0.35122185
 -0.36367254 -0.36737114 -0.5        -0.50038627 -0.61884033 -0.72772863
 -0.5001842  -0.61741538 -0.72818037 -0.27576162 -0.27542421 -0.28960978
 -0.28939089 -0.74641863 -0.7389559  -0.74161117 -0.74815961 -0.74037019
 -0.74055388]
25
[(0.49845948791714634, -0.28466157422324534), (0.49914523130034555, -0.35277207710011554), (0.46491368814729567, -0.35297508055235904), (0.408096029919447, -0.3689446041426925), (0.3553570667433837, -0.3778372105868816), (0.5342232922899873, -0.35122184810126583), (0.5916942048331423, -0.36367253509781344), (0.6439931737629461, -0.36737114499424617), (0.5, -0.5), (0.4784335431530497, -0.500386266973533),

In [5]:
POSE = "Second-Position"
allDistanceVectors = []
for sample in Coords:
    xCoords = sample[0::3]
    #print(xCoords)
    yPos = sample[1::3]
    yCoords = [-y for y in yPos] # need to make it negative to be consistent - the mean skeleton's y saved the negative coords.
    SampleTuples = list(zip(xCoords, yCoords))
    #print(SampleTuples)
    longZip = list(zip(MeanTuples, SampleTuples))
    #print(longZip)
    distanceVector = []
    for ((x1,y1),(x2,y2)) in longZip:
        d = calcDistance(x1,y1,x2,y2)
        #print(d)
        distanceVector.append(d)
    allDistanceVectors.append(distanceVector)

print(allDistanceVectors)
print(len(allDistanceVectors))
print(len(allDistanceVectors[0]))
np.save("DataForSVM/"+POSE+"DistanceVectorsPerSample.npy", allDistanceVectors)

[[0.009995297831501049, 0.009442411152905146, 0.008285006478503876, 0.005651379793631563, 0.008760831341647254, 0.009203426710374782, 0.006876954273493184, 0.012681437098772147, 0.0, 0.0010211466333700705, 0.008442371758506559, 0.013494857831382073, 0.002776333143952138, 0.012945638086410432, 0.0117936995293324, 0.013149081168995654, 0.013237978282255569, 0.011265558317561207, 0.01213330039621718, 0.014872405060537752, 0.016054542231081166, 0.010648247448221074, 0.01213641676305614, 0.011946772300210481, 0.013183485343508234], [0.013372384306673153, 0.011879449757236631, 0.011035416416979745, 0.018821604798898547, 0.012089704195631424, 0.01130549565020415, 0.0063318118500244915, 0.012705244869588845, 0.0, 0.0010017603012489708, 0.008354716320432753, 0.011194302049781205, 0.0028206737697969737, 0.012852489719603531, 0.009931685180979236, 0.015855882646302735, 0.015817719268190034, 0.01464089830630544, 0.015423730269733682, 0.013956132825806962, 0.014699249459201601, 0.008490760066767614

In [6]:
#These features are mixing coordinates and distances: 100 features per sample: 75 + 25
PoseAugFeaturesX = []
for i in range(len(Coords)):
    newList = list(Coords[i])+list(allDistanceVectors[i])
    PoseAugFeaturesX.append(newList)
    
print(PoseAugFeaturesX[0])
print(len(PoseAugFeaturesX[0]))
np.save("DataForSVM/"+POSE+"AugMeanDistFeaturesX.npy", PoseAugFeaturesX)

[0.504087, 0.27640099999999995, 0.871369, 0.504064, 0.34471199999999996, 0.896496, 0.465276, 0.34469799999999995, 0.853331, 0.41001499999999996, 0.363629, 0.845416, 0.357001, 0.36923199999999995, 0.8737929999999999, 0.54075, 0.34473299999999996, 0.8989119999999999, 0.598005, 0.366405, 0.838935, 0.642992, 0.380013, 0.824734, 0.5, 0.5, 0.7578539999999999, 0.477498, 0.499977, 0.7784749999999999, 0.4774729999999999, 0.62533, 0.859626, 0.461076, 0.739622, 0.913381, 0.5244599999999999, 0.502628, 0.7836759999999999, 0.524509, 0.628026, 0.878459, 0.5448799999999999, 0.73694, 0.856903, 0.493888, 0.262876, 0.9354049999999999, 0.5101479999999999, 0.26286099999999996, 0.9526509999999999, 0.4856579999999999, 0.27920399999999995, 0.856854, 0.5224449999999999, 0.27917299999999995, 0.893133, 0.5796399999999999, 0.758907, 0.739439, 0.579608, 0.753381, 0.702353, 0.54082, 0.750585, 0.821857, 0.42647599999999997, 0.7587769999999999, 0.746069, 0.42846799999999996, 0.750663, 0.684232, 0.463214, 0.753358, 0.